In [1]:
%cd C:/Users/sylee/Documents/GitHub/Final_2Kyoungh/data

C:\Users\sylee\Documents\GitHub\Final_2Kyoungh\data


In [2]:
import requests
import json
import pandas as pd

## 기본 세팅 부분
##종웅님 
headers = {"X-Naver-Client-Id": "EmdlUXsc4CuKNm_kxYKS",
           "X-Naver-Client-Secret":"dSLoIEmMgy"}

url = "https://openapi.naver.com/v1/datalab/shopping/categories"
def change_index(df, column_name):
    df = df.set_index("period")
    df.columns = [column_name]
    return df


In [3]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "골프", "param": [ "50000029"]},
          {"name": "기타스포츠용품", "param": [ "50000022"]},
          {"name": "낚시", "param": [ "50000163"]},
          {"name": "농구", "param": [ "50000038"]},
          {"name": "등산", "param": [ "50000027"]},
          {"name": "배구", "param": [ "50000039"]},
          {"name": "배드민턴", "param": [ "50000041"]},
          {"name": "볼링", "param": [ "50000045"]},
          {"name": "스케이트/보드/롤러", "param": [ "50000034"]},
          {"name": "스포츠액세서리", "param": [ "50000053"]},
          {"name": "야구", "param": [ "50000037"]},
          {"name": "오토바이/스쿠터", "param": [ "50000035"]},
          {"name": "축구", "param": [ "50000036"]},
          {"name": "캠핑", "param": [ "50000028"]},
          {"name": "탁구", "param": [ "50000040"]},
          {"name": "테니스", "param": [ "50000042"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
        {"name": "스포츠/레저", "param": [ "50000007"]},
        {"name": "헬스", "param": [ "50000030"]},
    ],
    "device":i,
    "gender":"",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "스포츠/레저")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "헬스")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='전체'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='전체'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='전체'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_a=pd.concat(df_list)
df_all_a.drop(["스포츠/레저"], axis=1, inplace=True)
df_all_a

,헬스,골프,기타스포츠용품,낚시,농구,등산,배구,배드민턴,볼링,스케이트/보드/롤러,스포츠액세서리,야구,오토바이/스쿠터,축구,캠핑,탁구,테니스,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,
2021-05-10,1.48247,6.31725,0.09527,2.79473,0.25168,4.58666,0.02251,0.22583,0.07876,0.30320,0.52287,0.28512,1.14932,0.69483,8.34848,0.06805,0.21610,PC+MO,전체,전체
2021-05-11,1.37974,5.81143,0.09847,2.65686,0.23979,4.29745,0.02083,0.21844,0.07840,0.28794,0.53522,0.25975,1.15128,0.67154,7.54099,0.06630,0.21354,PC+MO,전체,전체
2021-05-12,1.29958,5.63061,0.09660,2.44880,0.23948,4.20788,0.02035,0.23131,0.07250,0.29947,0.61209,0.24904,1.15924,0.66031,7.06668,0.06657,0.20051,PC+MO,전체,전체
2021-05-13,1.23625,5.46029,0.08971,2.17152,0.22751,4.14106,0.02088,0.21997,0.06673,0.29529,0.75785,0.22982,1.20081,0.59756,6.41095,0.06331,0.18982,PC+MO,전체,전체
2021-05-14,1.13501,5.25423,0.08570,1.93332,0.20525,3.77158,0.01712,0.19975,0.05776,0.24404,0.71739,0.21928,1.07214,0.55277,5.59236,0.05811,0.16968,PC+MO,전체,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,0.78453,4.44150,0.04954,2.18098,0.14688,2.70282,0.02802,0.32109,0.06596,0.08732,0.45902,0.20451,0.82154,0.97230,3.63736,0.09658,0.25218,MO,전체,전체
2023-06-30,0.72292,3.88554,0.04579,1.77062,0.13517,2.43338,0.02419,0.27870,0.06662,0.07664,0.39778,0.18729,0.74664,0.84826,3.06877,0.08689,0.22377,MO,전체,전체
2023-07-01,0.78165,4.99018,0.05035,1.66602,0.17659,3.01041,0.02912,0.32256,0.06851,0.10774,0.60266,0.25421,0.79464,1.10556,3.83817,0.09166,0.26756,MO,전체,전체


In [4]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "골프", "param": [ "50000029"]},
          {"name": "기타스포츠용품", "param": [ "50000022"]},
          {"name": "낚시", "param": [ "50000163"]},
          {"name": "농구", "param": [ "50000038"]},
          {"name": "등산", "param": [ "50000027"]},
          {"name": "배구", "param": [ "50000039"]},
          {"name": "배드민턴", "param": [ "50000041"]},
          {"name": "볼링", "param": [ "50000045"]},
          {"name": "스케이트/보드/롤러", "param": [ "50000034"]},
          {"name": "스포츠액세서리", "param": [ "50000053"]},
          {"name": "야구", "param": [ "50000037"]},
          {"name": "오토바이/스쿠터", "param": [ "50000035"]},
          {"name": "축구", "param": [ "50000036"]},
          {"name": "캠핑", "param": [ "50000028"]},
          {"name": "탁구", "param": [ "50000040"]},
          {"name": "테니스", "param": [ "50000042"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
        {"name": "스포츠/레저", "param": [ "50000007"]},
        {"name": "헬스", "param": [ "50000030"]},
    ],
    "device":i,
    "gender":"f",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "스포츠/레저")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "헬스")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='여성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='여성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='여성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_b=pd.concat(df_list)
df_all_b.drop(["스포츠/레저"], axis=1, inplace=True)
df_all_b

,헬스,골프,기타스포츠용품,낚시,농구,등산,배구,배드민턴,볼링,스케이트/보드/롤러,스포츠액세서리,야구,오토바이/스쿠터,축구,캠핑,탁구,테니스,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,
2021-05-10,1.71393,8.00292,0.08330,0.58382,0.18260,6.33252,0.02205,0.26941,0.05566,0.30532,0.51124,0.13551,0.38573,0.43874,9.73500,0.05495,0.25046,PC+MO,여성,전체
2021-05-11,1.60071,7.48618,0.09027,0.58537,0.19192,5.83649,0.02085,0.26436,0.05151,0.28826,0.53705,0.12006,0.40575,0.43734,8.83838,0.05169,0.24720,PC+MO,여성,전체
2021-05-12,1.47207,7.40930,0.09245,0.53947,0.18580,5.63276,0.02121,0.28895,0.04479,0.28219,0.67397,0.11728,0.42153,0.45294,8.02787,0.05441,0.24562,PC+MO,여성,전체
2021-05-13,1.41518,7.25015,0.07810,0.50003,0.16718,5.57903,0.02133,0.26836,0.04416,0.27930,0.87537,0.10613,0.45557,0.40581,7.45878,0.04712,0.24206,PC+MO,여성,전체
2021-05-14,1.27713,6.92821,0.07963,0.43319,0.14976,5.07502,0.01879,0.25596,0.04174,0.24048,0.86990,0.10491,0.37832,0.36332,6.29944,0.05046,0.20618,PC+MO,여성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,1.04093,6.39769,0.05761,0.48526,0.13250,4.02391,0.04097,0.46771,0.05610,0.08028,0.65029,0.13356,0.28495,1.03214,4.09244,0.10653,0.32535,MO,여성,전체
2023-06-30,0.97707,5.50380,0.05658,0.37561,0.11647,3.64954,0.03630,0.39870,0.05155,0.07352,0.58889,0.12787,0.24640,0.86809,3.49148,0.08565,0.28989,MO,여성,전체
2023-07-01,1.06927,6.94730,0.05879,0.38227,0.16884,4.64920,0.04261,0.45632,0.05434,0.10507,0.93922,0.18973,0.31272,1.19090,4.71933,0.09028,0.33896,MO,여성,전체


In [5]:
devices=['','pc','mo']

## 대분류 카테고리 설정
for i in devices:
    categories = [
          {"name": "골프", "param": [ "50000029"]},
          {"name": "기타스포츠용품", "param": [ "50000022"]},
          {"name": "낚시", "param": [ "50000163"]},
          {"name": "농구", "param": [ "50000038"]},
          {"name": "등산", "param": [ "50000027"]},
          {"name": "배구", "param": [ "50000039"]},
          {"name": "배드민턴", "param": [ "50000041"]},
          {"name": "볼링", "param": [ "50000045"]},
          {"name": "스케이트/보드/롤러", "param": [ "50000034"]},
          {"name": "스포츠액세서리", "param": [ "50000053"]},
          {"name": "야구", "param": [ "50000037"]},
          {"name": "오토바이/스쿠터", "param": [ "50000035"]},
          {"name": "축구", "param": [ "50000036"]},
          {"name": "캠핑", "param": [ "50000028"]},
          {"name": "탁구", "param": [ "50000040"]},
          {"name": "테니스", "param": [ "50000042"]}
      ]
## API 요청 값 작성, 디지털/가전 = result[0], 패션의류 = result[1]
    data = {
    "startDate": "2021-05-10",
    "endDate": "2023-07-03",
    "timeUnit": "date",
    "category": [
        {"name": "스포츠/레저", "param": [ "50000007"]},
        {"name": "헬스", "param": [ "50000030"]},
    ],
    "device":i,
    "gender":"m",
    "ages":[]
    }
## 데이터 정제 및 merge 파트
    response = requests.post(url, headers=headers, json=data)

    results = response.json()["results"]


    df_1 = change_index(pd.DataFrame(results[0]["data"]), "스포츠/레저")
    df_2 = change_index(pd.DataFrame(results[1]["data"]), "헬스")
##
    df_all_i = pd.merge(df_1, df_2, left_on="period", right_on="period")
## 반복문 통해서 카테고리에 설정된 값, 불러오고 저장, 합치기
    for category in categories:
        if len(data["category"]) > 2:
            data["category"].pop(2)
        data["category"].append(category)

        result = requests.post(url, headers=headers, json=data)

        results = result.json()["results"]

        df = change_index(pd.DataFrame(results[2]["data"]), category["name"])

        df_all_i = pd.merge(df_all_i, df, left_on="period", right_on="period")
    if i == "":
        df_all_1 = df_all_i
        df_all_1['유형']='PC+MO'
        df_all_1['성별']='남성'
        df_all_1['연령']='전체'
    if i == "pc":
        df_all_2 = df_all_i
        df_all_2['유형']='PC'
        df_all_2['성별']='남성'
        df_all_2['연령']='전체'
    if i == "mo":
        df_all_3 = df_all_i
        df_all_3['유형']='MO'
        df_all_3['성별']='남성'
        df_all_3['연령']='전체'
##    
df_list=[df_all_1,df_all_2,df_all_3]
df_all_c=pd.concat(df_list)
df_all_c.drop(["스포츠/레저"], axis=1, inplace=True)
df_all_c

,헬스,골프,기타스포츠용품,낚시,농구,등산,배구,배드민턴,볼링,스케이트/보드/롤러,스포츠액세서리,야구,오토바이/스쿠터,축구,캠핑,탁구,테니스,유형,성별,연령
period,,,,,,,,,,,,,,,,,,,,
2021-05-10,2.94181,12.42359,0.20665,9.21826,0.65798,7.76624,0.04442,0.39319,0.22174,0.64417,1.22244,0.81776,3.63347,1.84318,18.58243,0.16288,0.47235,PC+MO,남성,전체
2021-05-11,2.69182,11.39687,0.20479,8.75928,0.61217,7.37221,0.03790,0.39082,0.21715,0.62445,1.20637,0.75551,3.62899,1.78245,16.57593,0.15501,0.44578,PC+MO,남성,전체
2021-05-12,2.53546,10.84167,0.20683,8.12973,0.63607,7.39899,0.03695,0.40646,0.21387,0.68291,1.36068,0.72628,3.65264,1.72913,15.76840,0.15858,0.41404,PC+MO,남성,전체
2021-05-13,2.39240,10.40590,0.19156,7.13621,0.60353,7.26720,0.03779,0.38091,0.19287,0.66261,1.64035,0.67063,3.72105,1.56443,13.89049,0.14549,0.37628,PC+MO,남성,전체
2021-05-14,2.16792,9.95935,0.17924,6.33699,0.53278,6.56646,0.02762,0.33732,0.15789,0.53402,1.50935,0.64245,3.26408,1.43992,12.07439,0.12534,0.36341,PC+MO,남성,전체
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-29,1.67769,9.86502,0.13014,8.57593,0.43510,5.93209,0.06027,0.73197,0.20620,0.25203,1.03665,0.66333,3.05835,2.50021,9.74238,0.25504,0.64670,MO,남성,전체
2023-06-30,1.54239,8.86003,0.11629,6.95736,0.40848,5.27582,0.04927,0.64218,0.21513,0.21735,0.86251,0.60199,2.81906,2.21746,8.10060,0.24548,0.57302,MO,남성,전체
2023-07-01,1.71276,11.67679,0.13327,6.51642,0.51100,6.41774,0.06229,0.73875,0.21870,0.30071,1.29210,0.79371,2.94535,2.79525,9.74992,0.25588,0.69031,MO,남성,전체


In [6]:
df_list=[df_all_a,df_all_b,df_all_c]
df_all=pd.concat(df_list)

## 엑셀에 저장
df_all.to_excel("MCLS_06_all_fm_all.xlsx", sheet_name="click ratio")
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7064 entries, 2021-05-10 to 2023-07-03
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   헬스          7064 non-null   float64
 1   골프          7064 non-null   float64
 2   기타스포츠용품     7064 non-null   float64
 3   낚시          7064 non-null   float64
 4   농구          7064 non-null   float64
 5   등산          7064 non-null   float64
 6   배구          7064 non-null   float64
 7   배드민턴        7064 non-null   float64
 8   볼링          7064 non-null   float64
 9   스케이트/보드/롤러  7064 non-null   float64
 10  스포츠액세서리     7064 non-null   float64
 11  야구          7064 non-null   float64
 12  오토바이/스쿠터    7064 non-null   float64
 13  축구          7064 non-null   float64
 14  캠핑          7064 non-null   float64
 15  탁구          7064 non-null   float64
 16  테니스         7064 non-null   float64
 17  유형          7064 non-null   object 
 18  성별          7064 non-null   object 
 19  연령          7064 